# Explorando tendencias de mercado de Airbnb

La ciudad de Nueva York tiene una variedad de anuncios de Airbnb para satisfacer la alta demanda de alojamiento temporal para viajeros, con diferentes niveles de precios, tipos de habitaciones y ubicaciones.

La ciudad de Nueva York, es una de las ciudades más visitadas del mundo. Hay muchos anuncios de Airbnb en la ciudad de Nueva York para satisfacer la alta demanda de alojamiento temporal para viajeros, que puede durar desde unas pocas noches hasta muchos meses. En este proyecto, analizaremos más de cerca el mercado de Airbnb de Nueva York combinando datos de varios tipos de archivos como .csv, .tsv y .xlsx.

Recuerde que los archivos CSV, TSV y Excel son tres formatos comunes para almacenar datos. Tiene a su disposición [tres archivos](https://tajamar365.sharepoint.com/:f:/s/3405-MasterIA2024-2025/EoXMgjAJLLNJqWDyKDig8kABSUkpWj_HGQjl267qNnHS9g?e=foxjFc) que contienen datos sobre los anuncios de Airbnb de 2019:  

- `airbnb_price.csv `: Este es un archivo CSV que contiene datos sobre precios y ubicaciones de anuncios de Airbnb.  
  1. `listing_id`: identificador único del anuncio  
  2. `price`: precio del anuncio por noche en USD  
  3. `nbhood_full`: nombre del distrito y del barrio donde se encuentra el anuncio  
    

- `airbnb_room_type.xlsx`: Este es un archivo Excel que contiene datos sobre las descripciones de los anuncios de Airbnb y los tipos de habitaciones.  
  1. `listing_id`: identificador único del anuncio  
  2. `description`: descripción del anuncio     
  3. `room_type`: Airbnb tiene tres tipos de habitaciones: habitaciones compartidas, habitaciones privadas y casas o apartamentos completos 


- `airbnb_last_review.tsv`: Este es un archivo TSV que contiene datos sobre los nombres de los anfitriones de Airbnb y las fechas de revisión.  
  1. `listing_id`: identificador único del anuncio  
  2. `host_name`:  nombre del anfitrión del anuncio  
  3. `last_review`: fecha en la que se revisó el anuncio por última vez   

Como consultor que trabaja para una empresa inmobiliaria emergente, ha recopilado datos de anuncios de Airbnb de varias fuentes para investigar el mercado de alquileres a corto plazo en Nueva York. Analizará estos datos para brindar información sobre habitaciones privadas a la empresa inmobiliaria.

- ¿Cuáles son las fechas de las primeras y las últimas reseñas? Almacene estos valores como dos variables independientes con los nombres que prefiera.
- ¿Cuántos de los anuncios son habitaciones privadas? Guárdelo en cualquier variable.
- ¿Cuál es el precio promedio de los anuncios? Redondee a los dos decimales más cercanos y guárdelo en una variable.
- Combine las nuevas variables en un DataFrame llamado review_dates con cuatro columnas en el siguiente orden: first_reviewed, last_reviewed, nb_private_rooms y avg_price. El DataFrame solo debe contener una fila de valores.

Cargar los archivos como DataFrames (DF):

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Airbnb").getOrCreate()

# Cargar el archivo CSV en un DF
df_price = spark.read.option("sep", ",") \
               .option("header", "true") \
               .option("inferSchema", "true") \
               .csv("dbfs:/FileStore/PySpark-Data-Cleansing/Caso_3/airbnb_price.csv")

# Mostrar el esquema y las primeras filas del DF
df_price.printSchema()
df_price.show(5, truncate=False)  # Mostrar solamente las 5 primeras líneas

root
 |-- listing_id: integer (nullable = true)
 |-- price: string (nullable = true)
 |-- nbhood_full: string (nullable = true)

+----------+-----------+-------------------------+
|listing_id|price      |nbhood_full              |
+----------+-----------+-------------------------+
|2595      |225 dollars|Manhattan, Midtown       |
|3831      |89 dollars |Brooklyn, Clinton Hill   |
|5099      |200 dollars|Manhattan, Murray Hill   |
|5178      |79 dollars |Manhattan, Hell's Kitchen|
|5238      |150 dollars|Manhattan, Chinatown     |
+----------+-----------+-------------------------+
only showing top 5 rows



Exportar el archivo `XLSX` a `CSV` desde la aplicación de Excel e importarlo a `DBFS`. Después, leer el nuevo `CSV` en un DF.

> A tener en cuenta, normalmente, los archivos `CSV` delimitan sus valores con el carácter `,` (como el caso anterior), sin embargo, en este caso se debe notar que el carácter separador es `;`.

In [0]:
# Cargar el archivo CSV en un DF
df_room_type = spark.read.option("sep", ";") \
               .option("header", "true") \
               .option("inferSchema", "true") \
               .csv("dbfs:/FileStore/PySpark-Data-Cleansing/Caso_3/airbnb_room_type.csv")

df_room_type.printSchema()
df_room_type.show(5, truncate=False)

root
 |-- listing_id: string (nullable = true)
 |-- description: string (nullable = true)
 |-- room_type: string (nullable = true)

+----------+-----------------------------------------+---------------+
|listing_id|description                              |room_type      |
+----------+-----------------------------------------+---------------+
|2595      |Skylit Midtown Castle                    |Entire home/apt|
|3831      |Cozy Entire Floor of Brownstone          |Entire home/apt|
|5099      |Large Cozy 1 BR Apartment In Midtown East|Entire home/apt|
|5178      |Large Furnished Room Near B'way          |private room   |
|5238      |Cute & Cozy Lower East Side 1 bdrm       |Entire home/apt|
+----------+-----------------------------------------+---------------+
only showing top 5 rows



In [0]:
# Cargar el archivo TSV en un DF
df_last_review = spark.read.option("sep", "\t") \
               .option("header", "true") \
               .option("inferSchema", "true") \
               .csv("dbfs:/FileStore/PySpark-Data-Cleansing/Caso_3/airbnb_last_review.tsv")

df_last_review.printSchema()
df_last_review.show(5, truncate=False)

root
 |-- listing_id: integer (nullable = true)
 |-- host_name: string (nullable = true)
 |-- last_review: string (nullable = true)

+----------+-----------+------------+
|listing_id|host_name  |last_review |
+----------+-----------+------------+
|2595      |Jennifer   |May 21 2019 |
|3831      |LisaRoxanne|July 05 2019|
|5099      |Chris      |June 22 2019|
|5178      |Shunichi   |June 24 2019|
|5238      |Ben        |June 09 2019|
+----------+-----------+------------+
only showing top 5 rows



Convertir los DF's a Pandas:

In [0]:
import pandas as pd

# Convertir los DataFrames de PySpark a Pandas
df_price_pd = df_price.toPandas()
df_room_type_pd = df_room_type.toPandas()
df_last_review_pd = df_last_review.toPandas()

Unir los tres DF's en uno solo:

In [0]:
# Asegúrate de que 'listing_id' tenga el mismo tipo en todos los DataFrames
df_price_pd['listing_id'] = df_price_pd['listing_id'].astype(str)
df_room_type_pd['listing_id'] = df_room_type_pd['listing_id'].astype(str)
df_last_review_pd['listing_id'] = df_last_review_pd['listing_id'].astype(str)

# Realizar las uniones
df_airbnb_pd = pd.merge(df_price_pd, df_room_type_pd, on="listing_id")
df_airbnb_pd = pd.merge(df_airbnb_pd, df_last_review_pd, on="listing_id")

# Ver los primeros registros del DataFrame combinado
df_airbnb_pd.head()

,listing_id,price,nbhood_full,description,room_type,host_name,last_review
0,2595,225 dollars,"Manhattan, Midtown",Skylit Midtown Castle,Entire home/apt,Jennifer,May 21 2019
1,3831,89 dollars,"Brooklyn, Clinton Hill",Cozy Entire Floor of Brownstone,Entire home/apt,LisaRoxanne,July 05 2019
2,5099,200 dollars,"Manhattan, Murray Hill",Large Cozy 1 BR Apartment In Midtown East,Entire home/apt,Chris,June 22 2019
3,5178,79 dollars,"Manhattan, Hell's Kitchen",Large Furnished Room Near B'way,private room,Shunichi,June 24 2019
4,5238,150 dollars,"Manhattan, Chinatown",Cute & Cozy Lower East Side 1 bdrm,Entire home/apt,Ben,June 09 2019


### ¿Cuáles son las fechas de las primeras y las últimas reseñas? Almacene estos valores como dos variables independientes con los nombres que prefiera.

In [0]:
# Obtener la primera y última fecha de reseña
first_reviewed = df_airbnb_pd["last_review"].min()
last_reviewed = df_airbnb_pd["last_review"].max()

# Mostrar los resultados
print(f"La primera reseña fue hecha el {first_review.strftime('%b %d %Y')}")
print(f"La reseña más reciente fue hecha el {last_review.strftime('%b %d %Y')}")

La primera reseña fue hecha el Jan 01 2019
La reseña más reciente fue hecha el Jul 09 2019


### ¿Cuántos de los anuncios son habitaciones privadas? Guárdelo en cualquier variable.

In [0]:
# Obtener la suma de los registros cuyo tipo de habitación sea 'private room'
n_private_rooms = (df_airbnb_pd["room_type"].str.lower() == "private room").sum()

print(f"Hay {n_private_rooms} habitaciones privadas.")

Hay 11282 habitaciones privadas.


### ¿Cuál es el precio promedio de los anuncios? Redondee a los dos decimales más cercanos y guárdelo en una variable.

In [0]:
# Eliminar el texto " dollars" y convertir a tipo numérico
df_airbnb_pd['price'] = df_airbnb_pd['price'].replace(r'[^\d.]', '', regex=True).astype(float)

# Calcular el precio promedio
avg_price = df_airbnb_pd["price"].mean()

# Imprimir el resultado con formato de dos decimales
print(f"El precio promedio de los anuncios es {'{:.2f}'.format(avg_price)} dólares.")

El precio promedio de los anuncios es 141.78 dólares.


### Combine las nuevas variables en un DataFrame llamado review_dates con cuatro columnas en el siguiente orden: first_reviewed, last_reviewed, nb_private_rooms y avg_price. El DataFrame solo debe contener una fila de valores.

In [0]:
review_dates = pd.DataFrame(
    {
        "first_reviewed": [first_review],
        "last_reviewed": [last_review],
        "n_private_rooms": [n_private_rooms],
        "avg_price": [round(avg_price, 2)]
    }
)

review_dates

,first_reviewed,last_reviewed,n_private_rooms,avg_price
0,2019-01-01,2019-07-09,11282,141.78
